# 크롤링해야 되는 것
#### -> 자산총계, 부채총계, 자본총계 = 재무상태표
#### -> 매출액, 영업이익, 당기순이익(분기순이익) = 포괄손익계산서
api_key = 'ff12d16bfd2cb5ddef36494b71c419fda68b304d'


- 해당 분기보고서가 없을 때 예외처리를 생각해보기

### 기업들의 고유번호 받아오기

In [13]:
year = int(input("찾고자 하는 연도를 입력하시오: "))
print("\n")

anchor = int(input('''보고서 종류를 입력하시오(1,2,3,4 입력)
          1: 1분기 보고서
          2: 반기 보고서
          3: 3반기 보고서
          4: 사업보고서\n 입력: '''))
print("\n")

output_sheet_name = input('''시트 이름을 작성해주세요: ''')

if anchor == 1:
  report_num = "11013"
  excel_title = str(year) + " .1Q 경쟁사 실적 상세현황(직전분기 대비)"
  bs_first_col = str(int(str(year)[2:])-1) + "년말"
  bs_second_col = str(year)[2:] + ".1Q"
  cis_first_col = str(int(str(year)[2:])-1) + ".1Q"
  cis_second_col = bs_second_col

elif anchor == 2:
  report_num = "11012"
  excel_title = str(year) + " .2Q 경쟁사 실적 상세현황(직전분기 대비)"
  bs_first_col = str(int(str(year)[2:])-1) + "년말"
  bs_second_col = str(year)[2:] + ".2Q"
  cis_first_col = str(int(str(year)[2:])-1) + ".2Q"
  cis_second_col = bs_second_col

elif anchor == 3:
  report_num = "11014"
  excel_title = str(year) + " .3Q 경쟁사 실적 상세현황(직전분기 대비)"
  bs_first_col = str(int(str(year)[2:])-1) + "년말"
  bs_second_col = str(year)[2:] + ".3Q"
  cis_first_col = str(int(str(year)[2:])-1) + ".3Q"
  cis_second_col = bs_second_col

elif anchor == 4:
  report_num = "11011"
  excel_title = str(year) + " .4Q 경쟁사 실적 상세현황(직전분기 대비)"
  bs_first_col = str(int(str(year)[2:])-1) + "년말"
  bs_second_col = str(year)[2:] + ".4Q"
  cis_first_col = str(int(str(year)[2:])-1) + ".4Q"
  cis_second_col = bs_second_col

print(report_num)
print(excel_title)
print(bs_first_col)
print(bs_second_col)
print(cis_first_col)
print(cis_second_col)
print(type(year))


찾고자 하는 연도를 입력하시오: 2024


보고서 종류를 입력하시오(1,2,3,4 입력)
          1: 1분기 보고서
          2: 반기 보고서
          3: 3반기 보고서
          4: 사업보고서
 입력: 1


시트 이름을 작성해주세요: outputfile
11013
2024 .1Q 경쟁사 실적 상세현황(직전분기 대비)
23년말
24.1Q
23.1Q
24.1Q
<class 'int'>


In [ ]:
!pip install dart_fss

### 1. 패키지 설치 및 import + 비상장, 상장 기업 정보 가져오기

In [ ]:
import dart_fss as dart
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import zipfile
import io
from bs4 import BeautifulSoup
api_key = 'ff12d16bfd2cb5ddef36494b71c419fda68b304d'
dart.set_api_key(api_key=api_key)

# 상장+비상장 기업명 크롤링
corp_list = dart.api.filings.get_corp_code()
corp_df = pd.DataFrame.from_dict(corp_list)
# corp_df = corp_df.dropna(subset = 'stock_code').sort_values('modify_date',ascending=False).reset_index(drop=True)
corp_df = pd.DataFrame.from_dict(corp_list).sort_values('modify_date', ascending=False).reset_index(drop=True)
corp_df['done_YN'] = "N"
corp_df.columns = ['고유번호', '회사명', '종목코드', '수정날짜', 'done_YN']
corp_df.drop(columns = ['done_YN'], inplace = True)
display(corp_df)

,고유번호,회사명,종목코드,수정날짜
0,01891269,글로벌푸드그룹,None,20250121
1,01003253,신세계프라퍼티,None,20250121
2,01458161,티엑스알로보틱스,None,20250121
3,01173163,이에스알켄달스퀘어,None,20250121
4,01497887,풍산디에이케이,None,20250121
...,...,...,...,...
109410,00417547,스피어헤드,None,20170630
109411,00274298,신한산전,None,20170630
109412,00330886,쏘텍코리아,None,20170630
109413,00309017,씨아이브레인,None,20170630


### 2. 특정 기업 필터링

In [ ]:
# 주어진 company_dic
company_dic = {"SGA": "049470", "SGA솔루션즈":"184230", "안랩":"053800", "시큐브":"131090", "윈스":"136540", "이글루":"067920",
               "한컴위드":"054920", "네오리진": "094860", "케이사인":"192250", "이스트소프트":"047560", "라온시큐어": "042510","파수":"150900",
               "한국정보인증": "053300", "지란지교시큐리티":"208350","수산아이앤티":"050960", "지니언스": "263860", "드림시큐리티": "203650",
               "모니터랩":"434480", "파이오링크":"170790", "시큐센":"232830", "시큐레터": "418250",
               "다우기술":"023590", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820", "소프트센":"032680",
               "대신정보통신":"020180", "쌍용정보통신":"010280", "에스넷": "038680", '샌즈랩': "411080"
               }

# company_dic = {"SGA": "049470", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820",
#                 "쌍용정보통신":"010280", "토마토시스템": "393210", "오파스넷": "173130", "오픈베이스": "049480"}

# company_dic의 종목코드 리스트
target_stock_codes = list(company_dic.values())

# DataFrame에서 종목코드 필터링
filtered_df = corp_df[corp_df['종목코드'].isin(target_stock_codes)]


# company_dic의 keys에 따라 순서를 맞춘 DataFrame 생성
sorted_filtered_df = pd.DataFrame(columns=filtered_df.columns)

for key in company_dic.keys():
    stock_code = company_dic[key]
    matching_row = filtered_df[filtered_df['종목코드'] == stock_code]
    if not matching_row.empty:
        sorted_filtered_df = pd.concat([sorted_filtered_df, matching_row])

# 결과 출력
first_df = sorted_filtered_df.iloc[:2]
sequ_eye = corp_df[corp_df['회사명']=='시큐아이']
rest_df = sorted_filtered_df.iloc[2:]
sorted_filtered_df = pd.concat([first_df, sequ_eye,rest_df], ignore_index=True)
display(sorted_filtered_df)

,고유번호,회사명,종목코드,수정날짜
0,00351579,SGA,049470,20240801
1,00988364,SGA솔루션즈,184230,20240801
2,00294511,시큐아이,None,20230314
3,00298270,안랩,053800,20230206
4,00402110,시큐브,131090,20240829
5,00868705,윈스,136540,20221206
6,00364847,이글루,067920,20221129
7,00363592,한컴위드,054920,20230331
8,00599106,네오리진,094860,20221202
9,01003040,케이사인,192250,20240327


### 3. API 요청을 보내고 JSON 데이터를 받아 DataFrame으로 변환하는 함수

In [ ]:
def Frame(url, items, item_names, params):
    """
    url : json형태로 요청하는 주소
    items : 반환되는 데이터들의 key를 가진 리스트
    item_names :  데이터프레임을 만들때 컬럼명 리스트
    params : url 요청시 필수값으로 들어가는 인자들을 가진 딕셔너리
    """
    res = requests.get(url, params)
    json_data = res.json()
    json_dict = json.loads(res.text)
    data = []
    if json_dict['status'] == "000":  # 오류 없이 정상적으로 데이터가 있다면
        for line in json_dict['list']:
            data.append([])
            for itm in items:
                if itm in line.keys():
                    data[-1].append(line[itm])
                else:
                    data[-1].append('')
    df = pd.DataFrame(data, columns=item_names)
    return df

### 4. 단일 회사 재무제표 조회 함수
- crtfc_key = 개인 api 키
- corp_code = 기업 고유번호
- bsns_year = 해당 연도
- reprt_code =  보고서 종류(표 참고)
- fs_div = 재무제표 구분(연결 = "CFS" or 개별 = "OFS")

| 보고서 종류  | reprt_code 값 |
|-------------|--------------|
| 사업보고서  | `11011` |
| 반기보고서  | `11012` |
| 1분기보고서 | `11013` |
| 3분기보고서 | `11014` |

In [ ]:
def get_단일회사재무제표(crtfc_key, corp_code, bsns_year, reprt_code, fs_div = "CFS"):
    items = ["rcept_no","reprt_code","bsns_year","corp_code","sj_div","sj_nm",
             "account_id","account_nm","account_detail","thstrm_nm", "thstrm_amount",
             "thstrm_add_amount","frmtrm_nm","frmtrm_amount", "frmtrm_q_nm","frmtrm_q_amount",
             "frmtrm_add_amount","bfefrmtrm_nm", "bfefrmtrm_amount","ord"]
    item_names = ["접수번호","보고서코드","사업연도","고유번호","재무제표구분", "재무제표명",
                  "계정ID","계정명","계정상세","당기명","당기금액", "당기누적금액","전기명","전기금액","전기명(분/반기)",
                  "전기금액(분/반기)","전기누적금액","전전기명","전전기금액", "계정과목정렬순서"]
    params = {'crtfc_key':crtfc_key, 'corp_code':corp_code, 'bsns_year':bsns_year, 'reprt_code':reprt_code, 'fs_div':fs_div}
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?"
    df = Frame(url, items, item_names, params)
    return df

### 5. 전체 기업 재무제표 수집
- 2024 반기 정상 작동
- 2024 1분기 정상 작동
- 2024 3분기 대신정보통신 빠짐 -> 없어도 되게 함

In [ ]:
corp_code = list(sorted_filtered_df['고유번호'])
corp_code[0]

'00351579'

In [ ]:
import json


# corp_code = sorted_filtered_df['고유번호']
final_reprt_df = get_단일회사재무제표(api_key, corp_code[0], 2024, '11014', fs_div = "OFS")
#반복
from tqdm import tqdm
for k in tqdm(range(1, len(sorted_filtered_df))):
    reprt_df = get_단일회사재무제표(api_key, corp_code[k], 2024, '11014', fs_div = "OFS")
    final_reprt_df = pd.concat([final_reprt_df, reprt_df], axis = 0)

100%|██████████| 32/32 [00:12<00:00,  2.53it/s]


In [ ]:
final_reprt_df

,접수번호,보고서코드,사업연도,고유번호,재무제표구분,재무제표명,계정ID,계정명,계정상세,당기명,당기금액,당기누적금액,전기명,전기금액,전기명(분/반기),전기금액(분/반기),전기누적금액,전전기명,전전기금액,계정과목정렬순서
0,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 28 기 3분기말,41594765796,,제 27 기말,67327743710,,,,,,7
1,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 28 기 3분기말,9858840611,,제 27 기말,15515655313,,,,,,8
2,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 28 기 3분기말,3892510036,,제 27 기말,6450695754,,,,,,9
3,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,당기손익-공정가치금융자산,-,제 28 기 3분기말,553532400,,제 27 기말,628492695,,,,,,10
4,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_CurrentTaxAssets,당기법인세자산,-,제 28 기 3분기말,25046940,,제 27 기말,25386480,,,,,,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,20241114000984,11014,2024,01242241,SCE,자본변동표,ifrs-full_IssueOfEquity,유상증자,자본 [구성요소]|기타자본구성요소 [구성요소],제 21 기 3분기,-253478296,,,,제 20 기 3분기,,,,,7
83,20241114000984,11014,2024,01242241,SCE,자본변동표,ifrs-full_IssueOfEquity,유상증자,자본 [구성요소]|자본금 [구성요소],제 21 기 3분기,15340000,,,,제 20 기 3분기,309523800,,,,7
84,20241114000984,11014,2024,01242241,SCE,자본변동표,ifrs-full_IssueOfEquity,유상증자,자본 [구성요소]|자본잉여금 [구성요소],제 21 기 3분기,851738296,,,,제 20 기 3분기,30556221150,,,,7
85,20241114000984,11014,2024,01242241,SCE,자본변동표,ifrs-full_ProfitLoss,당기순이익(손실),별도재무제표 [member],제 21 기 3분기,-686318451,,,,제 20 기 3분기,-914693447,,,,8


# 개별 제무제표

### 6. 재무제표별 데이터 분리

In [ ]:
def sep(x, df):
    df = df[df['재무제표구분'] == x]
    return df

final_BS = sep('BS', final_reprt_df) #재무상태표
final_CIS = sep('CIS', final_reprt_df) #포괄손익계산서
final_CF = sep('CF', final_reprt_df) #현금흐름표
final_SCE = sep('SCE', final_reprt_df) #자본변동표
final_IS = sep('IS', final_reprt_df) #손익계산서

#### 재무상태표에서 자산총계, 자본총계, 부채총계 가져오기

In [ ]:
final_CIS.columns

Index(['접수번호', '보고서코드', '사업연도', '고유번호', '재무제표구분', '재무제표명', '계정ID', '계정명',
       '계정상세', '당기명', '당기금액', '당기누적금액', '전기명', '전기금액', '전기명(분/반기)',
       '전기금액(분/반기)', '전기누적금액', '전전기명', '전전기금액', '계정과목정렬순서'],
      dtype='object')

In [ ]:
final_BS

,접수번호,보고서코드,사업연도,고유번호,재무제표구분,재무제표명,계정ID,계정명,계정상세,당기명,당기금액,당기누적금액,전기명,전기금액,전기명(분/반기),전기금액(분/반기),전기누적금액,전전기명,전전기금액,계정과목정렬순서
0,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 28 기 3분기말,41594765796,,제 27 기말,67327743710,,,,,,7
1,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 28 기 3분기말,9858840611,,제 27 기말,15515655313,,,,,,8
2,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 28 기 3분기말,3892510036,,제 27 기말,6450695754,,,,,,9
3,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,당기손익-공정가치금융자산,-,제 28 기 3분기말,553532400,,제 27 기말,628492695,,,,,,10
4,20241114002244,11014,2024,00351579,BS,재무상태표,ifrs-full_CurrentTaxAssets,당기법인세자산,-,제 28 기 3분기말,25046940,,제 27 기말,25386480,,,,,,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,20241114000984,11014,2024,01242241,BS,재무상태표,ifrs-full_OtherCurrentLiabilities,기타 유동부채,-,제 21 기 3분기말,3944884148,,제 20 기말,258876143,,,,,,33
25,20241114000984,11014,2024,01242241,BS,재무상태표,ifrs-full_TradeAndOtherCurrentPayables,매입채무,-,제 21 기 3분기말,1146200000,,제 20 기말,4712170046,,,,,,34
26,20241114000984,11014,2024,01242241,BS,재무상태표,ifrs-full_Liabilities,부채총계,-,제 21 기 3분기말,6144691693,,제 20 기말,5743080985,,,,,,35
27,20241114000984,11014,2024,01242241,BS,재무상태표,ifrs-full_NoncurrentLiabilities,비유동부채,-,제 21 기 3분기말,11759082,,제 20 기말,25875487,,,,,,36


In [ ]:
bs_info_df = final_BS[(final_BS['계정명'] == '자산총계') | (final_BS['계정명'] == '자산 총계')
| (final_BS['계정명'] == '부채총계') | (final_BS['계정명'] == '부채 총계') | (final_BS['계정명'] == '자본총계') |
                      (final_BS['계정명'] == '자본 총계')][['고유번호', '재무제표명', "계정명", '전전기명', '전전기금액','전기명', '전기금액', '당기명', '당기금액'
                      ]]
display(bs_info_df)
print(bs_info_df.info())

bs_info_df = bs_info_df.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [bs_info_df.columns[-1]] + list(bs_info_df.columns[1:-1]) + [bs_info_df.columns[0]]
bs_info_df = bs_info_df[cols]
bs_info_df['계정명'] = bs_info_df['계정명'].str.replace(r'\s+', '', regex=True)
bs_info_df

,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액
0,00351579,재무상태표,자산총계,,,제 27 기말,67327743710,제 28 기 3분기말,41594765796
23,00351579,재무상태표,자본총계,,,제 27 기말,37600775039,제 28 기 3분기말,35882336715
35,00351579,재무상태표,부채총계,,,제 27 기말,29726968671,제 28 기 3분기말,5712429081
0,00988364,재무상태표,자산총계,,,제 11 기말,64381173529,제 12 기 3분기말,69562898876
21,00988364,재무상태표,자본총계,,,제 11 기말,45559699601,제 12 기 3분기말,48589550286
...,...,...,...,...,...,...,...,...,...
26,00264635,재무상태표,자본총계,,,제 25 기말,82876201912,제 26 기 3분기말,79665000026
43,00264635,재무상태표,부채총계,,,제 25 기말,135929911327,제 26 기 3분기말,102608655006
0,01242241,재무상태표,자산총계,,,제 20 기말,46362311664,제 21 기 3분기말,53618295318
17,01242241,재무상태표,자본총계,,,제 20 기말,40619230679,제 21 기 3분기말,47473603625


<class 'pandas.core.frame.DataFrame'>
Index: 96 entries, 0 to 26
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   고유번호    96 non-null     object
 1   재무제표명   96 non-null     object
 2   계정명     96 non-null     object
 3   전전기명    96 non-null     object
 4   전전기금액   96 non-null     object
 5   전기명     96 non-null     object
 6   전기금액    96 non-null     object
 7   당기명     96 non-null     object
 8   당기금액    96 non-null     object
dtypes: object(9)
memory usage: 9.6+ KB
None


,회사명,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호
0,SGA,재무상태표,자산총계,,,제 27 기말,67327743710,제 28 기 3분기말,41594765796,00351579
1,SGA,재무상태표,자본총계,,,제 27 기말,37600775039,제 28 기 3분기말,35882336715,00351579
2,SGA,재무상태표,부채총계,,,제 27 기말,29726968671,제 28 기 3분기말,5712429081,00351579
3,SGA솔루션즈,재무상태표,자산총계,,,제 11 기말,64381173529,제 12 기 3분기말,69562898876,00988364
4,SGA솔루션즈,재무상태표,자본총계,,,제 11 기말,45559699601,제 12 기 3분기말,48589550286,00988364
...,...,...,...,...,...,...,...,...,...,...
91,에스넷,재무상태표,자본총계,,,제 25 기말,82876201912,제 26 기 3분기말,79665000026,00264635
92,에스넷,재무상태표,부채총계,,,제 25 기말,135929911327,제 26 기 3분기말,102608655006,00264635
93,샌즈랩,재무상태표,자산총계,,,제 20 기말,46362311664,제 21 기 3분기말,53618295318,01242241
94,샌즈랩,재무상태표,자본총계,,,제 20 기말,40619230679,제 21 기 3분기말,47473603625,01242241


In [ ]:
total_comp_set = set(sorted_filtered_df['회사명'].unique())
bs_comp_set = set(bs_info_df['회사명'].unique())
print(total_comp_set - bs_comp_set)

{'대신정보통신'}


In [ ]:
기업명_순서 = list(sorted_filtered_df['회사명'])
계정명_순서 = ['자산총계', '부채총계', '자본총계']


# 기업명과 계정명에 따라 모든 조합을 포함하는 데이터프레임 생성
expected_rows = pd.DataFrame([(company, account) for company in 기업명_순서 for account in 계정명_순서],
                             columns=['회사명', '계정명'])

# 기존 데이터와 병합, 없는 데이터는 '업로드 없음'으로 채우기
bs_info_df = expected_rows.merge(bs_info_df, on=['회사명', '계정명'], how='left').fillna({'값': '업로드 없음'})

# '회사명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
bs_info_df['회사명'] = pd.Categorical(bs_info_df['회사명'], categories=기업명_순서, ordered=True)
bs_info_df['계정명'] = pd.Categorical(bs_info_df['계정명'], categories=계정명_순서, ordered=True)

# 정렬 적용
bs_info_df = bs_info_df.sort_values(by=['회사명', '계정명']).reset_index(drop=True)

bs_info_df['행 개수'] = bs_info_df.groupby('회사명')['회사명'].transform('count')
display(bs_info_df)


<ipython-input-74-ceb8c56f5f26>:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bs_info_df['행 개수'] = bs_info_df.groupby('회사명')['회사명'].transform('count')


,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,재무상태표,,,제 27 기말,67327743710,제 28 기 3분기말,41594765796,00351579,3
1,SGA,부채총계,재무상태표,,,제 27 기말,29726968671,제 28 기 3분기말,5712429081,00351579,3
2,SGA,자본총계,재무상태표,,,제 27 기말,37600775039,제 28 기 3분기말,35882336715,00351579,3
3,SGA솔루션즈,자산총계,재무상태표,,,제 11 기말,64381173529,제 12 기 3분기말,69562898876,00988364,3
4,SGA솔루션즈,부채총계,재무상태표,,,제 11 기말,18821473928,제 12 기 3분기말,20973348590,00988364,3
...,...,...,...,...,...,...,...,...,...,...,...
94,에스넷,부채총계,재무상태표,,,제 25 기말,135929911327,제 26 기 3분기말,102608655006,00264635,3
95,에스넷,자본총계,재무상태표,,,제 25 기말,82876201912,제 26 기 3분기말,79665000026,00264635,3
96,샌즈랩,자산총계,재무상태표,,,제 20 기말,46362311664,제 21 기 3분기말,53618295318,01242241,3
97,샌즈랩,부채총계,재무상태표,,,제 20 기말,5743080985,제 21 기 3분기말,6144691693,01242241,3


#### 포괄손익계산서에서 매출액, 영업이익, 당기순이익(분기순이익) 가져오기

In [ ]:
# print(final_CIS.columns)
CIS_info_df = final_CIS[(final_CIS['계정명'] == "매출액") | (final_CIS['계정명'] == "수익(매출액)") |
                        (final_CIS['계정명'] == "영업수익") |(final_CIS['계정명'] == "영업이익(손실)") |
                         (final_CIS['계정명'] == "영업이익") | (final_CIS['계정명'] == "분기순이익(손실)") |
                          (final_CIS['계정명'] == "분기순이익") | (final_CIS['계정명'] == "당기순이익(손실)" )|
                           (final_CIS['계정명'] == "당기순이익") | (final_CIS['계정명'] == "반기순이익(손실)") | (final_CIS['계정명'] == "반기순이익")][['사업연도', '고유번호', '재무제표명', '계정명', '전전기명','전전기금액','전기명', '전기금액', '전기금액(분/반기)','전기누적금액',
                           '당기명','당기금액', '당기누적금액']]
# CIS_info_df.info()
# pd.set_option('display.max_rows', None)  # 모든 행 출력


CIS_info_df = CIS_info_df.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [CIS_info_df.columns[-1]] + list(CIS_info_df.columns[1:-1]) + [CIS_info_df.columns[0]]
CIS_info_df = CIS_info_df[cols]
# 문자열 앞뒤 공백 제거
CIS_info_df['계정명'] = CIS_info_df['계정명'].str.strip()
CIS_info_df['계정명'] = CIS_info_df['계정명'].str.replace(r'\s+', '', regex=True)

# 조건에 따라 값 변경
CIS_info_df['계정명'] = CIS_info_df['계정명'].apply(lambda x:
    '매출액' if '매출액' in x or '영업수익' in x else
    '영업이익' if '영업이익' in x else
    '당기순이익' if '순이익' in x else x
)

display(CIS_info_df)
print(CIS_info_df.info())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도
0,SGA,00351579,포괄손익계산서,영업이익,,,,,-2854599447,-6066333476,제 28 기 3분기,56130323,-1477077030,2024
1,SGA,00351579,포괄손익계산서,당기순이익,,,,,-2957740161,-6714005619,제 28 기 3분기,1720503408,-1294795237,2024
2,SGA,00351579,포괄손익계산서,매출액,,,,,13312697558,32871595528,제 28 기 3분기,8202788780,22009845615,2024
3,SGA솔루션즈,00988364,포괄손익계산서,영업이익,,,,,558112256,-914254734,제 12 기 3분기,21026277,-1338693634,2024
4,SGA솔루션즈,00988364,포괄손익계산서,당기순이익,,,,,700881892,-2064366596,제 12 기 3분기,-153349066,-961988190,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,에스넷,00264635,포괄손익계산서,당기순이익,,,,,1458129520,802133393,제 26 기 3분기,1662155889,-1226032763,2024
88,에스넷,00264635,포괄손익계산서,매출액,,,,,77538013232,165940442056,제 26 기 3분기,70859261583,177763061256,2024
89,샌즈랩,01242241,포괄손익계산서,영업이익,,,,,-1028180250,-2900552822,제 21 기 3분기,-819732005,-3822860658,2024
90,샌즈랩,01242241,포괄손익계산서,당기순이익,,,,,-914693447,-1781017714,제 21 기 3분기,-686318451,-2461992362,2024


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   회사명         92 non-null     object
 1   고유번호        92 non-null     object
 2   재무제표명       92 non-null     object
 3   계정명         92 non-null     object
 4   전전기명        92 non-null     object
 5   전전기금액       92 non-null     object
 6   전기명         92 non-null     object
 7   전기금액        92 non-null     object
 8   전기금액(분/반기)  92 non-null     object
 9   전기누적금액      92 non-null     object
 10  당기명         92 non-null     object
 11  당기금액        92 non-null     object
 12  당기누적금액      92 non-null     object
 13  사업연도        92 non-null     object
dtypes: object(14)
memory usage: 10.2+ KB
None


### 포괄손익계산서 대신 손익계산서를 작성하여 해당 정보를 못가져오는 경우가 있다

In [ ]:
CIS_info_df['행 개수'] = CIS_info_df.groupby('회사명')['회사명'].transform('count')
display(CIS_info_df[CIS_info_df['행 개수'] != 3])
print("매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------")
print(CIS_info_df[CIS_info_df['행 개수'] != 3]['회사명'].unique())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
45,수산아이앤티,00577016,포괄손익계산서,당기순이익,,,,,1013164288,3277419350,제 27 기 3분기,1336431641,4285360414,2024,1
85,쌍용정보통신,00138303,포괄손익계산서,당기순이익,,,,,269529439,3794211505,제 44 기 3분기,9355601500,10325651846,2024,1


매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------
['수산아이앤티' '쌍용정보통신']


### 손익계산서에서 해당 기업 정보를 가져와서 포괄손익계산서 데이터프레임에 붙여준다
- 그리고 다시 정보 뽑아내기

In [ ]:
missing_comp = list(CIS_info_df[CIS_info_df['행 개수'] != 3]['고유번호'].unique())
if len(missing_comp)>=1:
  for i in missing_comp:
    try:
      df2 = final_IS[final_IS['고유번호'] == i]
      final_CIS = final_CIS[final_CIS['고유번호'] != i]
      final_CIS = pd.concat([final_CIS, df2], ignore_index=True)
    except:
      print("오류발생!")
else:
  print("모든 기업의 매출액, 영업이익, 당기순이익 크롤링 완료")


# print(final_CIS.columns)
CIS_info_df = final_CIS[(final_CIS['계정명'] == "매출액") | (final_CIS['계정명'] == "수익(매출액)") |
                        (final_CIS['계정명'] == "영업수익") |(final_CIS['계정명'] == "영업이익(손실)") |
                         (final_CIS['계정명'] == "영업이익") | (final_CIS['계정명'] == "분기순이익(손실)") |
                          (final_CIS['계정명'] == "분기순이익") | (final_CIS['계정명'] == "당기순이익(손실)" )|
                           (final_CIS['계정명'] == "당기순이익") | (final_CIS['계정명'] == "반기순이익(손실)") | (final_CIS['계정명'] == "반기순이익")][['사업연도', '고유번호', '재무제표명', '계정명', '전전기명','전전기금액','전기명', '전기금액', '전기금액(분/반기)','전기누적금액',
                           '당기명','당기금액', '당기누적금액']]
# CIS_info_df.info()
# pd.set_option('display.max_rows', None)  # 모든 행 출력


CIS_info_df = CIS_info_df.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [CIS_info_df.columns[-1]] + list(CIS_info_df.columns[1:-1]) + [CIS_info_df.columns[0]]
CIS_info_df = CIS_info_df[cols]
# 문자열 앞뒤 공백 제거
CIS_info_df['계정명'] = CIS_info_df['계정명'].str.strip()
CIS_info_df['계정명'] = CIS_info_df['계정명'].str.replace(r'\s+', '', regex=True)

# 조건에 따라 값 변경
CIS_info_df['계정명'] = CIS_info_df['계정명'].apply(lambda x:
    '매출액' if '매출액' in x or '영업수익' in x else
    '영업이익' if '영업이익' in x else
    '당기순이익' if '순이익' in x else x
)

display(CIS_info_df)
print(CIS_info_df.info())



,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도
0,SGA,00351579,포괄손익계산서,영업이익,,,,,-2854599447,-6066333476,제 28 기 3분기,56130323,-1477077030,2024
1,SGA,00351579,포괄손익계산서,당기순이익,,,,,-2957740161,-6714005619,제 28 기 3분기,1720503408,-1294795237,2024
2,SGA,00351579,포괄손익계산서,매출액,,,,,13312697558,32871595528,제 28 기 3분기,8202788780,22009845615,2024
3,SGA솔루션즈,00988364,포괄손익계산서,영업이익,,,,,558112256,-914254734,제 12 기 3분기,21026277,-1338693634,2024
4,SGA솔루션즈,00988364,포괄손익계산서,당기순이익,,,,,700881892,-2064366596,제 12 기 3분기,-153349066,-961988190,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,수산아이앤티,00577016,손익계산서,당기순이익,,,,,1013164288,3277419350,제 27 기 3분기,1336431641,4285360414,2024
92,수산아이앤티,00577016,손익계산서,매출액,,,,,5737659811,16643622440,제 27 기 3분기,6058705081,18709432379,2024
93,쌍용정보통신,00138303,손익계산서,영업이익,,,,,-124976832,3616712589,제 44 기 3분기,8509151930,9529042426,2024
94,쌍용정보통신,00138303,손익계산서,당기순이익,,,,,269529439,3794211505,제 44 기 3분기,9355601500,10325651846,2024


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   회사명         96 non-null     object
 1   고유번호        96 non-null     object
 2   재무제표명       96 non-null     object
 3   계정명         96 non-null     object
 4   전전기명        96 non-null     object
 5   전전기금액       96 non-null     object
 6   전기명         96 non-null     object
 7   전기금액        96 non-null     object
 8   전기금액(분/반기)  96 non-null     object
 9   전기누적금액      96 non-null     object
 10  당기명         96 non-null     object
 11  당기금액        96 non-null     object
 12  당기누적금액      96 non-null     object
 13  사업연도        96 non-null     object
dtypes: object(14)
memory usage: 10.6+ KB
None


### 모든 기업이 매출액, 당기순이익, 영업이익 정보가 있는지 재확인

In [ ]:
CIS_info_df['행 개수'] = CIS_info_df.groupby('회사명')['회사명'].transform('count')
display(CIS_info_df[CIS_info_df['행 개수'] != 3])
print("매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------")
print(CIS_info_df[CIS_info_df['행 개수'] != 3]['회사명'].unique())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수


매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------
[]


### 재무제표, 포괄손익계산서 계정명 통일하기
- 금액 컬럼 데이터타입 숫자로 바꿔주기

In [ ]:
bs_info_df['계정명'].unique()

['자산총계', '부채총계', '자본총계']
Categories (3, object): ['자산총계' < '부채총계' < '자본총계']

In [ ]:
CIS_info_df['계정명'].unique()

array(['영업이익', '당기순이익', '매출액'], dtype=object)

In [ ]:
# df1의 '회사명' 순서를 기준으로 df2 정렬
bs_info_df['회사명'] = pd.Categorical(bs_info_df['회사명'], categories=sorted_filtered_df['회사명'], ordered=True)
bs_info_df = bs_info_df.sort_values('회사명').reset_index(drop=True)
display(bs_info_df)


CIS_info_df['회사명'] = pd.Categorical(CIS_info_df['회사명'], categories=sorted_filtered_df['회사명'], ordered=True)
CIS_info_df = CIS_info_df.sort_values('회사명').reset_index(drop=True)
display(CIS_info_df)


,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,재무상태표,,,제 27 기말,67327743710,제 28 기 3분기말,41594765796,00351579,3
1,SGA,부채총계,재무상태표,,,제 27 기말,29726968671,제 28 기 3분기말,5712429081,00351579,3
2,SGA,자본총계,재무상태표,,,제 27 기말,37600775039,제 28 기 3분기말,35882336715,00351579,3
3,SGA솔루션즈,자산총계,재무상태표,,,제 11 기말,64381173529,제 12 기 3분기말,69562898876,00988364,3
4,SGA솔루션즈,부채총계,재무상태표,,,제 11 기말,18821473928,제 12 기 3분기말,20973348590,00988364,3
...,...,...,...,...,...,...,...,...,...,...,...
94,에스넷,자산총계,재무상태표,,,제 25 기말,218806113239,제 26 기 3분기말,182273655032,00264635,3
95,에스넷,자본총계,재무상태표,,,제 25 기말,82876201912,제 26 기 3분기말,79665000026,00264635,3
96,샌즈랩,부채총계,재무상태표,,,제 20 기말,5743080985,제 21 기 3분기말,6144691693,01242241,3
97,샌즈랩,자산총계,재무상태표,,,제 20 기말,46362311664,제 21 기 3분기말,53618295318,01242241,3


,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
0,SGA,00351579,포괄손익계산서,영업이익,,,,,-2854599447,-6066333476,제 28 기 3분기,56130323,-1477077030,2024,3
1,SGA,00351579,포괄손익계산서,당기순이익,,,,,-2957740161,-6714005619,제 28 기 3분기,1720503408,-1294795237,2024,3
2,SGA,00351579,포괄손익계산서,매출액,,,,,13312697558,32871595528,제 28 기 3분기,8202788780,22009845615,2024,3
3,SGA솔루션즈,00988364,포괄손익계산서,영업이익,,,,,558112256,-914254734,제 12 기 3분기,21026277,-1338693634,2024,3
4,SGA솔루션즈,00988364,포괄손익계산서,당기순이익,,,,,700881892,-2064366596,제 12 기 3분기,-153349066,-961988190,2024,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,에스넷,00264635,포괄손익계산서,영업이익,,,,,1338292810,81028958,제 26 기 3분기,1865003793,-1848658607,2024,3
92,에스넷,00264635,포괄손익계산서,당기순이익,,,,,1458129520,802133393,제 26 기 3분기,1662155889,-1226032763,2024,3
93,샌즈랩,01242241,포괄손익계산서,당기순이익,,,,,-914693447,-1781017714,제 21 기 3분기,-686318451,-2461992362,2024,3
94,샌즈랩,01242241,포괄손익계산서,영업이익,,,,,-1028180250,-2900552822,제 21 기 3분기,-819732005,-3822860658,2024,3


In [ ]:
기업명_순서 = list(sorted_filtered_df['회사명'])
계정명_순서 = [ '매출액', '영업이익', '당기순이익']


# 기업명과 계정명에 따라 모든 조합을 포함하는 데이터프레임 생성
expected_rows = pd.DataFrame([(company, account) for company in 기업명_순서 for account in 계정명_순서],
                             columns=['회사명', '계정명'])

# 기존 데이터와 병합, 없는 데이터는 '업로드 없음'으로 채우기
CIS_info_df = expected_rows.merge(CIS_info_df, on=['회사명', '계정명'], how='left').fillna({'값': '업로드 없음'})

# '회사명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
CIS_info_df['회사명'] = pd.Categorical(CIS_info_df['회사명'], categories=기업명_순서, ordered=True)
CIS_info_df['계정명'] = pd.Categorical(CIS_info_df['계정명'], categories=계정명_순서, ordered=True)

# 정렬 적용
CIS_info_df = CIS_info_df.sort_values(by=['회사명', '계정명']).reset_index(drop=True)


display(CIS_info_df)

,회사명,계정명,고유번호,재무제표명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
0,SGA,매출액,00351579,포괄손익계산서,,,,,13312697558,32871595528,제 28 기 3분기,8202788780,22009845615,2024,3.0
1,SGA,영업이익,00351579,포괄손익계산서,,,,,-2854599447,-6066333476,제 28 기 3분기,56130323,-1477077030,2024,3.0
2,SGA,당기순이익,00351579,포괄손익계산서,,,,,-2957740161,-6714005619,제 28 기 3분기,1720503408,-1294795237,2024,3.0
3,SGA솔루션즈,매출액,00988364,포괄손익계산서,,,,,3570265028,7627159620,제 12 기 3분기,2176348394,6830828774,2024,3.0
4,SGA솔루션즈,영업이익,00988364,포괄손익계산서,,,,,558112256,-914254734,제 12 기 3분기,21026277,-1338693634,2024,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,에스넷,영업이익,00264635,포괄손익계산서,,,,,1338292810,81028958,제 26 기 3분기,1865003793,-1848658607,2024,3.0
95,에스넷,당기순이익,00264635,포괄손익계산서,,,,,1458129520,802133393,제 26 기 3분기,1662155889,-1226032763,2024,3.0
96,샌즈랩,매출액,01242241,포괄손익계산서,,,,,1269158953,2857611383,제 21 기 3분기,1792994254,3428228610,2024,3.0
97,샌즈랩,영업이익,01242241,포괄손익계산서,,,,,-1028180250,-2900552822,제 21 기 3분기,-819732005,-3822860658,2024,3.0


In [ ]:
# final_BS[final_BS['고유번호'] == "00620868"]
# BS_info_df


bs_nan_rows = bs_info_df[bs_info_df.isna().any(axis=1)]
cis_nan_rows = CIS_info_df[CIS_info_df.isna().any(axis=1)]
print(bs_nan_rows)
print(cis_nan_rows)

       회사명   계정명 재무제표명 전전기명 전전기금액  전기명 전기금액  당기명 당기금액 고유번호  행 개수
87  대신정보통신  자산총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
88  대신정보통신  부채총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
89  대신정보통신  자본총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
       회사명    계정명 고유번호 재무제표명 전전기명 전전기금액  전기명 전기금액 전기금액(분/반기) 전기누적금액  당기명 당기금액  \
87  대신정보통신    매출액  NaN   NaN  NaN   NaN  NaN  NaN        NaN    NaN  NaN  NaN   
88  대신정보통신   영업이익  NaN   NaN  NaN   NaN  NaN  NaN        NaN    NaN  NaN  NaN   
89  대신정보통신  당기순이익  NaN   NaN  NaN   NaN  NaN  NaN        NaN    NaN  NaN  NaN   

   당기누적금액 사업연도  행 개수  
87    NaN  NaN   NaN  
88    NaN  NaN   NaN  
89    NaN  NaN   NaN  


In [ ]:

bs_numeric_cols = ['전전기금액', '전기금액', '당기금액']
cis_numeric_cols = ['전전기금액', '전기금액', '전기금액(분/반기)', '전기누적금액', '당기금액', '당기누적금액']

# 숫자형 변환 (에러 발생 시 NaN 처리)
bs_info_df[bs_numeric_cols] = bs_info_df[bs_numeric_cols].apply(pd.to_numeric, errors='coerce')
CIS_info_df[cis_numeric_cols] = CIS_info_df[cis_numeric_cols].apply(pd.to_numeric, errors='coerce')

# 변환 후 데이터 타입 확인
print(bs_info_df.dtypes)
print(CIS_info_df.dtypes)

회사명      category
계정명      category
재무제표명      object
전전기명       object
전전기금액     float64
전기명        object
전기금액      float64
당기명        object
당기금액      float64
고유번호       object
행 개수        int64
dtype: object
회사명           category
계정명           category
고유번호            object
재무제표명           object
전전기명            object
전전기금액          float64
전기명             object
전기금액           float64
전기금액(분/반기)     float64
전기누적금액         float64
당기명             object
당기금액           float64
당기누적금액         float64
사업연도            object
행 개수           float64
dtype: object


In [ ]:
bs_info_df['재_type']= 'OFS'
CIS_info_df['재_type'] = 'OFS'

## 엑셀에 넣기
- bs_info_df
- CIS_info_df

# 연결 제무제표

### 6. 재무제표별 데이터 분리

In [ ]:
import json


# corp_code = sorted_filtered_df['고유번호']
final_reprt_df_conn = get_단일회사재무제표(api_key, corp_code[0], 2024, '11014', fs_div = "CFS")
#반복
from tqdm import tqdm
for k in tqdm(range(1, len(sorted_filtered_df))):
    reprt_df = get_단일회사재무제표(api_key, corp_code[k], 2024, '11014', fs_div = "CFS")
    final_reprt_df_conn = pd.concat([final_reprt_df_conn, reprt_df], axis = 0)

100%|██████████| 32/32 [00:11<00:00,  2.71it/s]


In [ ]:
def sep(x, df):
    df = df[df['재무제표구분'] == x]
    return df

final_BS_conn = sep('BS', final_reprt_df_conn) #재무상태표
final_CIS_conn = sep('CIS', final_reprt_df_conn) #포괄손익계산서
final_CF_conn = sep('CF', final_reprt_df_conn) #현금흐름표
final_SCE_conn = sep('SCE', final_reprt_df_conn) #자본변동표
final_IS_conn = sep('IS', final_reprt_df_conn) #손익계산서

#### 재무상태표에서 자산총계, 자본총계, 부채총계 가져오기

In [ ]:
final_CIS_conn.columns

Index(['접수번호', '보고서코드', '사업연도', '고유번호', '재무제표구분', '재무제표명', '계정ID', '계정명',
       '계정상세', '당기명', '당기금액', '당기누적금액', '전기명', '전기금액', '전기명(분/반기)',
       '전기금액(분/반기)', '전기누적금액', '전전기명', '전전기금액', '계정과목정렬순서'],
      dtype='object')

In [ ]:
final_BS_conn

,접수번호,보고서코드,사업연도,고유번호,재무제표구분,재무제표명,계정ID,계정명,계정상세,당기명,당기금액,당기누적금액,전기명,전기금액,전기명(분/반기),전기금액(분/반기),전기누적금액,전전기명,전전기금액,계정과목정렬순서
0,20241114002414,11014,2024,00988364,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 12 기 3분기말,105179866225,,제 11 기말,91846725203,,,,,,7
1,20241114002414,11014,2024,00988364,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 12 기 3분기말,19851531787,,제 11 기말,33289357236,,,,,,8
2,20241114002414,11014,2024,00988364,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 12 기 3분기말,6406554028,,제 11 기말,15381476512,,,,,,9
3,20241114002414,11014,2024,00988364,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,당기손익-공정가치측정금융자산,-,제 12 기 3분기말,1360199715,,제 11 기말,2937737105,,,,,,10
4,20241114002414,11014,2024,00988364,BS,재무상태표,ifrs-full_DerivativeFinancialAssets,파생상품자산,-,제 12 기 3분기말,286637440,,제 11 기말,0,,,,,,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,20241114002405,11014,2024,00264635,BS,재무상태표,dart_LongTermAccruedExpensesGross,미지급비용,-,제 26 기 3분기말,400977757,,제 25 기말,397145517,,,,,,58
50,20241114002405,11014,2024,00264635,BS,재무상태표,dart_PostemploymentBenefitObligations,순확정급여부채,-,제 26 기 3분기말,2488741321,,제 25 기말,847462171,,,,,,59
51,20241114002405,11014,2024,00264635,BS,재무상태표,ifrs-full_DeferredTaxLiabilities,이연법인세부채,-,제 26 기 3분기말,0,,제 25 기말,362047772,,,,,,60
52,20241114002405,11014,2024,00264635,BS,재무상태표,ifrs-full_NoncurrentLeaseLiabilities,리스부채,-,제 26 기 3분기말,3547609995,,제 25 기말,4005246809,,,,,,61


In [ ]:
bs_info_df_conn = final_BS_conn[(final_BS_conn['계정명'] == '자산총계') | (final_BS_conn['계정명'] == '자산 총계')
| (final_BS_conn['계정명'] == '부채총계') | (final_BS_conn['계정명'] == '부채 총계') | (final_BS_conn['계정명'] == '자본총계') |
                      (final_BS_conn['계정명'] == '자본 총계')][['고유번호', '재무제표명', "계정명", '전전기명', '전전기금액','전기명', '전기금액', '당기명', '당기금액'
                      ]]
display(bs_info_df_conn)
print(bs_info_df_conn.info())

bs_info_df_conn = bs_info_df_conn.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [bs_info_df_conn.columns[-1]] + list(bs_info_df_conn.columns[1:-1]) + [bs_info_df_conn.columns[0]]
bs_info_df_conn = bs_info_df_conn[cols]
bs_info_df_conn['계정명'] = bs_info_df_conn['계정명'].str.replace(r'\s+', '', regex=True)
bs_info_df_conn

,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액
0,00988364,재무상태표,자산총계,,,제 11 기말,91846725203,제 12 기 3분기말,105179866225
21,00988364,재무상태표,자본총계,,,제 11 기말,57069941362,제 12 기 3분기말,53103080513
39,00988364,재무상태표,부채총계,,,제 11 기말,34776783841,제 12 기 3분기말,52076785713
0,00298270,재무상태표,자산총계,,,제 29 기말,375384295785,제 30 기 3분기말,449019944524
18,00298270,재무상태표,자본총계,,,제 29 기말,284716086934,제 30 기 3분기말,363339365206
...,...,...,...,...,...,...,...,...,...
20,00138303,재무상태표,자본총계,,,제 43 기말,64984261170,제 44 기 3분기말,66753296541
37,00138303,재무상태표,부채총계,,,제 43 기말,136059779037,제 44 기 3분기말,173961351143
0,00264635,재무상태표,자산총계,,,제 25 기말,292747563284,제 26 기 3분기말,262814265766
26,00264635,재무상태표,자본총계,,,제 25 기말,107842565793,제 26 기 3분기말,101731611648


<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 0 to 47
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   고유번호    75 non-null     object
 1   재무제표명   75 non-null     object
 2   계정명     75 non-null     object
 3   전전기명    75 non-null     object
 4   전전기금액   75 non-null     object
 5   전기명     75 non-null     object
 6   전기금액    75 non-null     object
 7   당기명     75 non-null     object
 8   당기금액    75 non-null     object
dtypes: object(9)
memory usage: 7.9+ KB
None


,회사명,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호
0,SGA솔루션즈,재무상태표,자산총계,,,제 11 기말,91846725203,제 12 기 3분기말,105179866225,00988364
1,SGA솔루션즈,재무상태표,자본총계,,,제 11 기말,57069941362,제 12 기 3분기말,53103080513,00988364
2,SGA솔루션즈,재무상태표,부채총계,,,제 11 기말,34776783841,제 12 기 3분기말,52076785713,00988364
3,안랩,재무상태표,자산총계,,,제 29 기말,375384295785,제 30 기 3분기말,449019944524,00298270
4,안랩,재무상태표,자본총계,,,제 29 기말,284716086934,제 30 기 3분기말,363339365206,00298270
...,...,...,...,...,...,...,...,...,...,...
70,쌍용정보통신,재무상태표,자본총계,,,제 43 기말,64984261170,제 44 기 3분기말,66753296541,00138303
71,쌍용정보통신,재무상태표,부채총계,,,제 43 기말,136059779037,제 44 기 3분기말,173961351143,00138303
72,에스넷,재무상태표,자산총계,,,제 25 기말,292747563284,제 26 기 3분기말,262814265766,00264635
73,에스넷,재무상태표,자본총계,,,제 25 기말,107842565793,제 26 기 3분기말,101731611648,00264635


In [ ]:
total_comp_set = set(sorted_filtered_df['회사명'].unique())
bs_comp_set = set(bs_info_df_conn['회사명'].unique())
print(total_comp_set - bs_comp_set)

{'시큐센', 'SGA', '수산아이앤티', '대신정보통신', '샌즈랩', '시큐아이', '시큐브', '시큐레터'}


In [ ]:
기업명_순서 = list(sorted_filtered_df['회사명'])
계정명_순서 = ['자산총계', '부채총계', '자본총계']


# 기업명과 계정명에 따라 모든 조합을 포함하는 데이터프레임 생성
expected_rows = pd.DataFrame([(company, account) for company in 기업명_순서 for account in 계정명_순서],
                             columns=['회사명', '계정명'])

# 기존 데이터와 병합, 없는 데이터는 '업로드 없음'으로 채우기
bs_info_df_conn = expected_rows.merge(bs_info_df_conn, on=['회사명', '계정명'], how='left').fillna({'값': '업로드 없음'})

# '회사명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
bs_info_df_conn['회사명'] = pd.Categorical(bs_info_df_conn['회사명'], categories=기업명_순서, ordered=True)
bs_info_df_conn['계정명'] = pd.Categorical(bs_info_df_conn['계정명'], categories=계정명_순서, ordered=True)

# 정렬 적용
bs_info_df_conn = bs_info_df_conn.sort_values(by=['회사명', '계정명']).reset_index(drop=True)

bs_info_df_conn['행 개수'] = bs_info_df_conn.groupby('회사명')['회사명'].transform('count')
display(bs_info_df_conn)


<ipython-input-92-9352ae98a441>:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bs_info_df_conn['행 개수'] = bs_info_df_conn.groupby('회사명')['회사명'].transform('count')


,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,SGA,부채총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,SGA,자본총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,SGA솔루션즈,자산총계,재무상태표,,,제 11 기말,91846725203,제 12 기 3분기말,105179866225,00988364,3
4,SGA솔루션즈,부채총계,재무상태표,,,제 11 기말,34776783841,제 12 기 3분기말,52076785713,00988364,3
...,...,...,...,...,...,...,...,...,...,...,...
94,에스넷,부채총계,재무상태표,,,제 25 기말,184904997491,제 26 기 3분기말,161082654118,00264635,3
95,에스넷,자본총계,재무상태표,,,제 25 기말,107842565793,제 26 기 3분기말,101731611648,00264635,3
96,샌즈랩,자산총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
97,샌즈랩,부채총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


#### 포괄손익계산서에서 매출액, 영업이익, 당기순이익(분기순이익) 가져오기

In [ ]:
# print(final_CIS.columns)
CIS_info_df_conn = final_CIS_conn[(final_CIS_conn['계정명'] == "매출액") | (final_CIS_conn['계정명'] == "수익(매출액)") |
                        (final_CIS_conn['계정명'] == "영업수익") |(final_CIS_conn['계정명'] == "영업이익(손실)") |
                         (final_CIS_conn['계정명'] == "영업이익") | (final_CIS_conn['계정명'] == "분기순이익(손실)") |
                          (final_CIS_conn['계정명'] == "분기순이익") | (final_CIS_conn['계정명'] == "당기순이익(손실)" )|
                           (final_CIS_conn['계정명'] == "당기순이익") | (final_CIS_conn['계정명'] == "반기순이익(손실)") | (final_CIS_conn['계정명'] == "반기순이익")][['사업연도', '고유번호', '재무제표명', '계정명', '전전기명','전전기금액','전기명', '전기금액', '전기금액(분/반기)','전기누적금액',
                           '당기명','당기금액', '당기누적금액']]
# CIS_info_df.info()
# pd.set_option('display.max_rows', None)  # 모든 행 출력


CIS_info_df_conn = CIS_info_df_conn.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [CIS_info_df_conn.columns[-1]] + list(CIS_info_df_conn.columns[1:-1]) + [CIS_info_df_conn.columns[0]]
CIS_info_df_conn = CIS_info_df_conn[cols]
# 문자열 앞뒤 공백 제거
CIS_info_df_conn['계정명'] = CIS_info_df_conn['계정명'].str.strip()
CIS_info_df_conn['계정명'] = CIS_info_df_conn['계정명'].str.replace(r'\s+', '', regex=True)

# 조건에 따라 값 변경
CIS_info_df_conn['계정명'] = CIS_info_df_conn['계정명'].apply(lambda x:
    '매출액' if '매출액' in x or '영업수익' in x else
    '영업이익' if '영업이익' in x else
    '당기순이익' if '순이익' in x else x
)

display(CIS_info_df_conn)
print(CIS_info_df_conn.info())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도
0,SGA솔루션즈,00988364,포괄손익계산서,영업이익,,,,,-731657143,-2587519846,제 12 기 3분기,-1433533703,-5727938112,2024
1,SGA솔루션즈,00988364,포괄손익계산서,당기순이익,,,,,-649363896,-4087453538,제 12 기 3분기,-2075447426,-6376103904,2024
2,SGA솔루션즈,00988364,포괄손익계산서,매출액,,,,,11462942476,33238475249,제 12 기 3분기,9033069247,27032223888,2024
3,안랩,00298270,포괄손익계산서,영업이익,,,,,8323814572,14836695239,제 30 기 3분기,8528762859,12121229813,2024
4,안랩,00298270,포괄손익계산서,당기순이익,,,,,6081462149,17336935024,제 30 기 3분기,9057045194,14844359424,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,소프트센,00204226,포괄손익계산서,매출액,,,,,16161612992,42232153105,제 38 기 3분기,13261125570,40696237476,2024
69,쌍용정보통신,00138303,포괄손익계산서,당기순이익,,,,,4894122246,6038240909,제 44 기 3분기,3455114640,1605118428,2024
70,에스넷,00264635,포괄손익계산서,영업이익,,,,,3956155697,1355072112,제 26 기 3분기,2651881116,-2453157314,2024
71,에스넷,00264635,포괄손익계산서,당기순이익,,,,,3738034730,1630506482,제 26 기 3분기,1650134510,-4489152443,2024


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   회사명         73 non-null     object
 1   고유번호        73 non-null     object
 2   재무제표명       73 non-null     object
 3   계정명         73 non-null     object
 4   전전기명        73 non-null     object
 5   전전기금액       73 non-null     object
 6   전기명         73 non-null     object
 7   전기금액        73 non-null     object
 8   전기금액(분/반기)  73 non-null     object
 9   전기누적금액      73 non-null     object
 10  당기명         73 non-null     object
 11  당기금액        73 non-null     object
 12  당기누적금액      73 non-null     object
 13  사업연도        73 non-null     object
dtypes: object(14)
memory usage: 8.1+ KB
None


### 포괄손익계산서 대신 손익계산서를 작성하여 해당 정보를 못가져오는 경우가 있다

In [ ]:
CIS_info_df_conn['행 개수'] = CIS_info_df_conn.groupby('회사명')['회사명'].transform('count')
display(CIS_info_df_conn[CIS_info_df_conn['행 개수'] != 3])
print("매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------")
print(CIS_info_df_conn[CIS_info_df_conn['행 개수'] != 3]['회사명'].unique())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
69,쌍용정보통신,00138303,포괄손익계산서,당기순이익,,,,,4894122246,6038240909,제 44 기 3분기,3455114640,1605118428,2024,1


매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------
['쌍용정보통신']


### 손익계산서에서 해당 기업 정보를 가져와서 포괄손익계산서 데이터프레임에 붙여준다
- 그리고 다시 정보 뽑아내기

In [ ]:
missing_comp = list(CIS_info_df_conn[CIS_info_df_conn['행 개수'] != 3]['고유번호'].unique())
if len(missing_comp)>=1:
  for i in missing_comp:
    try:
      df2 = final_IS_conn[final_IS_conn['고유번호'] == i]
      final_CIS_conn = final_CIS_conn[final_CIS_conn['고유번호'] != i]
      final_CIS_conn = pd.concat([final_CIS_conn, df2], ignore_index=True)
    except:
      print("오류발생!")
else:
  print("모든 기업의 매출액, 영업이익, 당기순이익 크롤링 완료")


# print(final_CIS.columns)
CIS_info_df_conn = final_CIS_conn[(final_CIS_conn['계정명'] == "매출액") | (final_CIS_conn['계정명'] == "수익(매출액)") |
                        (final_CIS_conn['계정명'] == "영업수익") |(final_CIS_conn['계정명'] == "영업이익(손실)") |
                         (final_CIS_conn['계정명'] == "영업이익") | (final_CIS_conn['계정명'] == "분기순이익(손실)") |
                          (final_CIS_conn['계정명'] == "분기순이익") | (final_CIS_conn['계정명'] == "당기순이익(손실)" )|
                           (final_CIS_conn['계정명'] == "당기순이익") | (final_CIS_conn['계정명'] == "반기순이익(손실)") | (final_CIS_conn['계정명'] == "반기순이익")][['사업연도', '고유번호', '재무제표명', '계정명', '전전기명','전전기금액','전기명', '전기금액', '전기금액(분/반기)','전기누적금액',
                           '당기명','당기금액', '당기누적금액']]
# CIS_info_df.info()
# pd.set_option('display.max_rows', None)  # 모든 행 출력


CIS_info_df_conn = CIS_info_df_conn.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [CIS_info_df_conn.columns[-1]] + list(CIS_info_df_conn.columns[1:-1]) + [CIS_info_df_conn.columns[0]]
CIS_info_df_conn = CIS_info_df_conn[cols]
# 문자열 앞뒤 공백 제거
CIS_info_df_conn['계정명'] = CIS_info_df_conn['계정명'].str.strip()
CIS_info_df_conn['계정명'] = CIS_info_df_conn['계정명'].str.replace(r'\s+', '', regex=True)

# 조건에 따라 값 변경
CIS_info_df_conn['계정명'] = CIS_info_df_conn['계정명'].apply(lambda x:
    '매출액' if '매출액' in x or '영업수익' in x else
    '영업이익' if '영업이익' in x else
    '당기순이익' if '순이익' in x else x
)

display(CIS_info_df_conn)
print(CIS_info_df_conn.info())



,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도
0,SGA솔루션즈,00988364,포괄손익계산서,영업이익,,,,,-731657143,-2587519846,제 12 기 3분기,-1433533703,-5727938112,2024
1,SGA솔루션즈,00988364,포괄손익계산서,당기순이익,,,,,-649363896,-4087453538,제 12 기 3분기,-2075447426,-6376103904,2024
2,SGA솔루션즈,00988364,포괄손익계산서,매출액,,,,,11462942476,33238475249,제 12 기 3분기,9033069247,27032223888,2024
3,안랩,00298270,포괄손익계산서,영업이익,,,,,8323814572,14836695239,제 30 기 3분기,8528762859,12121229813,2024
4,안랩,00298270,포괄손익계산서,당기순이익,,,,,6081462149,17336935024,제 30 기 3분기,9057045194,14844359424,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,에스넷,00264635,포괄손익계산서,당기순이익,,,,,3738034730,1630506482,제 26 기 3분기,1650134510,-4489152443,2024
71,에스넷,00264635,포괄손익계산서,매출액,,,,,121896609141,279592806988,제 26 기 3분기,110450431606,287365896865,2024
72,쌍용정보통신,00138303,손익계산서,영업이익,,,,,1351961463,1598323769,제 44 기 3분기,2491144579,-1508822603,2024
73,쌍용정보통신,00138303,손익계산서,당기순이익,,,,,4894122246,6038240909,제 44 기 3분기,3455114640,1605118428,2024


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   회사명         75 non-null     object
 1   고유번호        75 non-null     object
 2   재무제표명       75 non-null     object
 3   계정명         75 non-null     object
 4   전전기명        75 non-null     object
 5   전전기금액       75 non-null     object
 6   전기명         75 non-null     object
 7   전기금액        75 non-null     object
 8   전기금액(분/반기)  75 non-null     object
 9   전기누적금액      75 non-null     object
 10  당기명         75 non-null     object
 11  당기금액        75 non-null     object
 12  당기누적금액      75 non-null     object
 13  사업연도        75 non-null     object
dtypes: object(14)
memory usage: 8.3+ KB
None


### 모든 기업이 매출액, 당기순이익, 영업이익 정보가 있는지 재확인

In [ ]:
CIS_info_df_conn['행 개수'] = CIS_info_df_conn.groupby('회사명')['회사명'].transform('count')
display(CIS_info_df_conn[CIS_info_df_conn['행 개수'] != 3])
print("매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------")
print(CIS_info_df_conn[CIS_info_df_conn['행 개수'] != 3]['회사명'].unique())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수


매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------
[]


### 재무제표, 포괄손익계산서 계정명 통일하기
- 금액 컬럼 데이터타입 숫자로 바꿔주기

In [ ]:
bs_info_df_conn['계정명'].unique()

['자산총계', '부채총계', '자본총계']
Categories (3, object): ['자산총계' < '부채총계' < '자본총계']

In [ ]:
CIS_info_df_conn['계정명'].unique()

array(['영업이익', '당기순이익', '매출액'], dtype=object)

In [ ]:
# df1의 '회사명' 순서를 기준으로 df2 정렬
bs_info_df_conn['회사명'] = pd.Categorical(bs_info_df_conn['회사명'], categories=sorted_filtered_df['회사명'], ordered=True)
bs_info_df_conn = bs_info_df_conn.sort_values('회사명').reset_index(drop=True)
display(bs_info_df_conn)


CIS_info_df_conn['회사명'] = pd.Categorical(CIS_info_df_conn['회사명'], categories=sorted_filtered_df['회사명'], ordered=True)
CIS_info_df_conn = CIS_info_df_conn.sort_values('회사명').reset_index(drop=True)
display(CIS_info_df_conn)


,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,SGA,부채총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,SGA,자본총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,SGA솔루션즈,자산총계,재무상태표,,,제 11 기말,91846725203,제 12 기 3분기말,105179866225,00988364,3
4,SGA솔루션즈,부채총계,재무상태표,,,제 11 기말,34776783841,제 12 기 3분기말,52076785713,00988364,3
...,...,...,...,...,...,...,...,...,...,...,...
94,에스넷,자산총계,재무상태표,,,제 25 기말,292747563284,제 26 기 3분기말,262814265766,00264635,3
95,에스넷,자본총계,재무상태표,,,제 25 기말,107842565793,제 26 기 3분기말,101731611648,00264635,3
96,샌즈랩,부채총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
97,샌즈랩,자산총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
0,SGA솔루션즈,00988364,포괄손익계산서,영업이익,,,,,-731657143,-2587519846,제 12 기 3분기,-1433533703,-5727938112,2024,3
1,SGA솔루션즈,00988364,포괄손익계산서,당기순이익,,,,,-649363896,-4087453538,제 12 기 3분기,-2075447426,-6376103904,2024,3
2,SGA솔루션즈,00988364,포괄손익계산서,매출액,,,,,11462942476,33238475249,제 12 기 3분기,9033069247,27032223888,2024,3
3,안랩,00298270,포괄손익계산서,영업이익,,,,,8323814572,14836695239,제 30 기 3분기,8528762859,12121229813,2024,3
4,안랩,00298270,포괄손익계산서,당기순이익,,,,,6081462149,17336935024,제 30 기 3분기,9057045194,14844359424,2024,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,쌍용정보통신,00138303,손익계산서,당기순이익,,,,,4894122246,6038240909,제 44 기 3분기,3455114640,1605118428,2024,3
71,쌍용정보통신,00138303,손익계산서,매출액,,,,,120112516791,307174798575,제 44 기 3분기,126983042844,276204573303,2024,3
72,에스넷,00264635,포괄손익계산서,매출액,,,,,121896609141,279592806988,제 26 기 3분기,110450431606,287365896865,2024,3
73,에스넷,00264635,포괄손익계산서,영업이익,,,,,3956155697,1355072112,제 26 기 3분기,2651881116,-2453157314,2024,3


In [ ]:
기업명_순서 = list(sorted_filtered_df['회사명'])
계정명_순서 = [ '매출액', '영업이익', '당기순이익']


# 기업명과 계정명에 따라 모든 조합을 포함하는 데이터프레임 생성
expected_rows = pd.DataFrame([(company, account) for company in 기업명_순서 for account in 계정명_순서],
                             columns=['회사명', '계정명'])

# 기존 데이터와 병합, 없는 데이터는 '업로드 없음'으로 채우기
CIS_info_df_conn = expected_rows.merge(CIS_info_df_conn, on=['회사명', '계정명'], how='left').fillna({'값': '업로드 없음'})

# '회사명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
CIS_info_df_conn['회사명'] = pd.Categorical(CIS_info_df_conn['회사명'], categories=기업명_순서, ordered=True)
CIS_info_df_conn['계정명'] = pd.Categorical(CIS_info_df_conn['계정명'], categories=계정명_순서, ordered=True)

# 정렬 적용
CIS_info_df_conn = CIS_info_df_conn.sort_values(by=['회사명', '계정명']).reset_index(drop=True)


display(CIS_info_df_conn)

,회사명,계정명,고유번호,재무제표명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
0,SGA,매출액,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SGA,영업이익,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SGA,당기순이익,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SGA솔루션즈,매출액,00988364,포괄손익계산서,,,,,11462942476,33238475249,제 12 기 3분기,9033069247,27032223888,2024,3.0
4,SGA솔루션즈,영업이익,00988364,포괄손익계산서,,,,,-731657143,-2587519846,제 12 기 3분기,-1433533703,-5727938112,2024,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,에스넷,영업이익,00264635,포괄손익계산서,,,,,3956155697,1355072112,제 26 기 3분기,2651881116,-2453157314,2024,3.0
95,에스넷,당기순이익,00264635,포괄손익계산서,,,,,3738034730,1630506482,제 26 기 3분기,1650134510,-4489152443,2024,3.0
96,샌즈랩,매출액,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,샌즈랩,영업이익,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# final_BS[final_BS['고유번호'] == "00620868"]
# BS_info_df


bs_nan_rows = bs_info_df_conn[bs_info_df_conn.isna().any(axis=1)]
cis_nan_rows = CIS_info_df_conn[CIS_info_df_conn.isna().any(axis=1)]
print(bs_nan_rows)
print(cis_nan_rows)

       회사명   계정명 재무제표명 전전기명 전전기금액  전기명 전기금액  당기명 당기금액 고유번호  행 개수
0      SGA  자산총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
1      SGA  부채총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
2      SGA  자본총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
6     시큐아이  자산총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
7     시큐아이  부채총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
8     시큐아이  자본총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
12     시큐브  자본총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
13     시큐브  자산총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
14     시큐브  부채총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
45  수산아이앤티  부채총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
46  수산아이앤티  자본총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
47  수산아이앤티  자산총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
60     시큐센  자본총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
61     시큐센  부채총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
62     시큐센  자산총계   NaN  N

In [ ]:

bs_numeric_cols = ['전전기금액', '전기금액', '당기금액']
cis_numeric_cols = ['전전기금액', '전기금액', '전기금액(분/반기)', '전기누적금액', '당기금액', '당기누적금액']

# 숫자형 변환 (에러 발생 시 NaN 처리)
bs_info_df_conn[bs_numeric_cols] = bs_info_df_conn[bs_numeric_cols].apply(pd.to_numeric, errors='coerce')
CIS_info_df_conn[cis_numeric_cols] = CIS_info_df_conn[cis_numeric_cols].apply(pd.to_numeric, errors='coerce')

# 변환 후 데이터 타입 확인
print(bs_info_df_conn.dtypes)
print(CIS_info_df_conn.dtypes)

회사명      category
계정명      category
재무제표명      object
전전기명       object
전전기금액     float64
전기명        object
전기금액      float64
당기명        object
당기금액      float64
고유번호       object
행 개수        int64
dtype: object
회사명           category
계정명           category
고유번호            object
재무제표명           object
전전기명            object
전전기금액          float64
전기명             object
전기금액           float64
전기금액(분/반기)     float64
전기누적금액         float64
당기명             object
당기금액           float64
당기누적금액         float64
사업연도            object
행 개수           float64
dtype: object


In [ ]:
bs_info_df_conn

,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,SGA,부채총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,SGA,자본총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,SGA솔루션즈,자산총계,재무상태표,,NaN,제 11 기말,9.184673e+10,제 12 기 3분기말,1.051799e+11,00988364,3
4,SGA솔루션즈,부채총계,재무상태표,,NaN,제 11 기말,3.477678e+10,제 12 기 3분기말,5.207679e+10,00988364,3
...,...,...,...,...,...,...,...,...,...,...,...
94,에스넷,자산총계,재무상태표,,NaN,제 25 기말,2.927476e+11,제 26 기 3분기말,2.628143e+11,00264635,3
95,에스넷,자본총계,재무상태표,,NaN,제 25 기말,1.078426e+11,제 26 기 3분기말,1.017316e+11,00264635,3
96,샌즈랩,부채총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
97,샌즈랩,자산총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [ ]:
bs_info_df_conn['재_type'] = 'CFS'
CIS_info_df_conn['재_type'] = 'CFS'

In [ ]:
bs_info_df_combined = pd.concat([bs_info_df, bs_info_df_conn], ignore_index=True)
CIS_info_df_combined = pd.concat([CIS_info_df, CIS_info_df_conn], ignore_index=True)
display(bs_info_df_combined)
display(CIS_info_df_combined)

,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수,재_type
0,SGA,자산총계,재무상태표,,NaN,제 27 기말,6.732774e+10,제 28 기 3분기말,4.159477e+10,00351579,3,OFS
1,SGA,부채총계,재무상태표,,NaN,제 27 기말,2.972697e+10,제 28 기 3분기말,5.712429e+09,00351579,3,OFS
2,SGA,자본총계,재무상태표,,NaN,제 27 기말,3.760078e+10,제 28 기 3분기말,3.588234e+10,00351579,3,OFS
3,SGA솔루션즈,자산총계,재무상태표,,NaN,제 11 기말,6.438117e+10,제 12 기 3분기말,6.956290e+10,00988364,3,OFS
4,SGA솔루션즈,부채총계,재무상태표,,NaN,제 11 기말,1.882147e+10,제 12 기 3분기말,2.097335e+10,00988364,3,OFS
...,...,...,...,...,...,...,...,...,...,...,...,...
193,에스넷,자산총계,재무상태표,,NaN,제 25 기말,2.927476e+11,제 26 기 3분기말,2.628143e+11,00264635,3,CFS
194,에스넷,자본총계,재무상태표,,NaN,제 25 기말,1.078426e+11,제 26 기 3분기말,1.017316e+11,00264635,3,CFS
195,샌즈랩,부채총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,CFS
196,샌즈랩,자산총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,CFS


,회사명,계정명,고유번호,재무제표명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수,재_type
0,SGA,매출액,00351579,포괄손익계산서,,NaN,,NaN,1.331270e+10,3.287160e+10,제 28 기 3분기,8.202789e+09,2.200985e+10,2024,3.0,OFS
1,SGA,영업이익,00351579,포괄손익계산서,,NaN,,NaN,-2.854599e+09,-6.066333e+09,제 28 기 3분기,5.613032e+07,-1.477077e+09,2024,3.0,OFS
2,SGA,당기순이익,00351579,포괄손익계산서,,NaN,,NaN,-2.957740e+09,-6.714006e+09,제 28 기 3분기,1.720503e+09,-1.294795e+09,2024,3.0,OFS
3,SGA솔루션즈,매출액,00988364,포괄손익계산서,,NaN,,NaN,3.570265e+09,7.627160e+09,제 12 기 3분기,2.176348e+09,6.830829e+09,2024,3.0,OFS
4,SGA솔루션즈,영업이익,00988364,포괄손익계산서,,NaN,,NaN,5.581123e+08,-9.142547e+08,제 12 기 3분기,2.102628e+07,-1.338694e+09,2024,3.0,OFS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,에스넷,영업이익,00264635,포괄손익계산서,,NaN,,NaN,3.956156e+09,1.355072e+09,제 26 기 3분기,2.651881e+09,-2.453157e+09,2024,3.0,CFS
194,에스넷,당기순이익,00264635,포괄손익계산서,,NaN,,NaN,3.738035e+09,1.630506e+09,제 26 기 3분기,1.650135e+09,-4.489152e+09,2024,3.0,CFS
195,샌즈랩,매출액,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CFS
196,샌즈랩,영업이익,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CFS


In [ ]:
bs_info_df_combined[bs_info_df_combined['회사명'] == 'SGA']

,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수,재_type
0,SGA,자산총계,재무상태표,,NaN,제 27 기말,6.732774e+10,제 28 기 3분기말,4.159477e+10,00351579,3,OFS
1,SGA,부채총계,재무상태표,,NaN,제 27 기말,2.972697e+10,제 28 기 3분기말,5.712429e+09,00351579,3,OFS
2,SGA,자본총계,재무상태표,,NaN,제 27 기말,3.760078e+10,제 28 기 3분기말,3.588234e+10,00351579,3,OFS
99,SGA,자산총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,CFS
100,SGA,부채총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,CFS
101,SGA,자본총계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,CFS


In [ ]:
columns_to_convert = ["전전기금액", "전기금액", "당기금액"]  # 변환할 컬럼 리스트
bs_info_df_combined[columns_to_convert] = bs_info_df_combined[columns_to_convert].fillna(0).astype(int)

columns_to_convert = ["전기금액(분/반기)", "전기누적금액", "당기금액", "당기누적금액"]  # 변환할 컬럼 리스트
CIS_info_df_combined[columns_to_convert] = CIS_info_df_combined[columns_to_convert].fillna(0).astype(int)

display(bs_info_df_combined)
display(CIS_info_df_combined)


,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수,재_type
0,SGA,자산총계,재무상태표,,0,제 27 기말,67327743710,제 28 기 3분기말,41594765796,00351579,3,OFS
1,SGA,부채총계,재무상태표,,0,제 27 기말,29726968671,제 28 기 3분기말,5712429081,00351579,3,OFS
2,SGA,자본총계,재무상태표,,0,제 27 기말,37600775039,제 28 기 3분기말,35882336715,00351579,3,OFS
3,SGA솔루션즈,자산총계,재무상태표,,0,제 11 기말,64381173529,제 12 기 3분기말,69562898876,00988364,3,OFS
4,SGA솔루션즈,부채총계,재무상태표,,0,제 11 기말,18821473928,제 12 기 3분기말,20973348590,00988364,3,OFS
...,...,...,...,...,...,...,...,...,...,...,...,...
193,에스넷,자산총계,재무상태표,,0,제 25 기말,292747563284,제 26 기 3분기말,262814265766,00264635,3,CFS
194,에스넷,자본총계,재무상태표,,0,제 25 기말,107842565793,제 26 기 3분기말,101731611648,00264635,3,CFS
195,샌즈랩,부채총계,NaN,NaN,0,NaN,0,NaN,0,NaN,3,CFS
196,샌즈랩,자산총계,NaN,NaN,0,NaN,0,NaN,0,NaN,3,CFS


,회사명,계정명,고유번호,재무제표명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수,재_type
0,SGA,매출액,00351579,포괄손익계산서,,NaN,,NaN,13312697558,32871595528,제 28 기 3분기,8202788780,22009845615,2024,3.0,OFS
1,SGA,영업이익,00351579,포괄손익계산서,,NaN,,NaN,-2854599447,-6066333476,제 28 기 3분기,56130323,-1477077030,2024,3.0,OFS
2,SGA,당기순이익,00351579,포괄손익계산서,,NaN,,NaN,-2957740161,-6714005619,제 28 기 3분기,1720503408,-1294795237,2024,3.0,OFS
3,SGA솔루션즈,매출액,00988364,포괄손익계산서,,NaN,,NaN,3570265028,7627159620,제 12 기 3분기,2176348394,6830828774,2024,3.0,OFS
4,SGA솔루션즈,영업이익,00988364,포괄손익계산서,,NaN,,NaN,558112256,-914254734,제 12 기 3분기,21026277,-1338693634,2024,3.0,OFS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,에스넷,영업이익,00264635,포괄손익계산서,,NaN,,NaN,3956155697,1355072112,제 26 기 3분기,2651881116,-2453157314,2024,3.0,CFS
194,에스넷,당기순이익,00264635,포괄손익계산서,,NaN,,NaN,3738034730,1630506482,제 26 기 3분기,1650134510,-4489152443,2024,3.0,CFS
195,샌즈랩,매출액,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,CFS
196,샌즈랩,영업이익,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,CFS


## 엑셀에 넣기
- bs_info_df
- CIS_info_df

In [ ]:
# import pandas as pd
# from openpyxl import load_workbook

# # 엑셀 파일 경로
# excel_path = "/content/drive/MyDrive/webcrawling/2023년도 2Q 경쟁사 실적현황 (SGA).xlsx"
# new_excel_path = "updated_excel.xlsx"  # 새 파일 경로
# sheet_name = "23.1Q_2305"

# # 기존 엑셀 파일 불러오기 (서식 유지)
# wb = load_workbook(excel_path)
# ws = wb[sheet_name]

# # 데이터프레임 로드 (bs_info_df, CIS_info_df는 미리 정의되어 있어야 함)
# # bs_info_df
# # CIS_info_df

# # 엑셀의 5행과 21행에 있는 회사명 찾기
# target_rows = [5, 21]  # openpyxl은 1-based index (pandas는 0-based index)
# for row in target_rows:
#     for col in range(1, ws.max_column + 1):  # 모든 열 탐색 (openpyxl은 1부터 시작)
#         company_name = ws.cell(row=row, column=col).value  # 회사명 가져오기

#         if company_name in bs_info_df["회사명"].values:
#             try:
#                 # 특정 계정명의 전기금액 & 당기금액 입력 (openpyxl은 1-based index)
#                 ws.cell(row=row + 2, column=col, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "자산총계"), "전기금액"].values[0])
#                 ws.cell(row=row + 3, column=col, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "부채총계"), "전기금액"].values[0])
#                 ws.cell(row=row + 4, column=col, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "자본총계"), "전기금액"].values[0])
#                 ws.cell(row=row + 2, column=col + 1, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "자산총계"), "당기금액"].values[0])
#                 ws.cell(row=row + 3, column=col + 1, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "부채총계"), "당기금액"].values[0])
#                 ws.cell(row=row + 4, column=col + 1, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "자본총계"), "당기금액"].values[0])

#                 ws.cell(row=row + 6, column=col, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "매출액"), "전기누적금액"].values[0])
#                 ws.cell(row=row + 7, column=col, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "영업이익"), "전기누적금액"].values[0])
#                 ws.cell(row=row + 9, column=col, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "당기순이익"), "전기누적금액"].values[0])
#                 ws.cell(row=row + 6, column=col + 1, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "매출액"), "당기누적금액"].values[0])
#                 ws.cell(row=row + 7, column=col + 1, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "영업이익"), "당기누적금액"].values[0])
#                 ws.cell(row=row + 9, column=col + 1, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "당기순이익"), "당기누적금액"].values[0])
#             except IndexError:
#                 print(f"데이터 누락: {company_name}의 일부 값이 없습니다.")

# # 수정된 데이터를 새 파일로 저장 (서식 유지)
# wb.save(new_excel_path)

# print(f"업데이트 완료: {new_excel_path} -> '{sheet_name}' 시트에 반영됨 (서식 유지됨)")


In [ ]:
bs_info_df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   회사명     198 non-null    category
 1   계정명     198 non-null    category
 2   재무제표명   171 non-null    object  
 3   전전기명    171 non-null    object  
 4   전전기금액   198 non-null    int64   
 5   전기명     171 non-null    object  
 6   전기금액    198 non-null    int64   
 7   당기명     171 non-null    object  
 8   당기금액    198 non-null    int64   
 9   고유번호    171 non-null    object  
 10  행 개수    198 non-null    int64   
 11  재_type  198 non-null    object  
dtypes: category(2), int64(4), object(6)
memory usage: 17.4+ KB


In [ ]:
# 변경할 매핑을 딕셔너리로 정의
replace_dict = {
    '시큐아이': '시큐아이(비상장)',
    '샌즈랩': '샌즈랩(기술특례)',
    '시큐레터': '시큐레터(기술특례)'
}

# replace 메서드를 활용하여 한 번에 변경
bs_info_df_combined['회사명'] = bs_info_df_combined['회사명'].replace(replace_dict)
CIS_info_df_combined['회사명'] = CIS_info_df_combined['회사명'].replace(replace_dict)

<ipython-input-110-bc07fd9c22ff>:9: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  bs_info_df_combined['회사명'] = bs_info_df_combined['회사명'].replace(replace_dict)
<ipython-input-110-bc07fd9c22ff>:10: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  CIS_info_df_combined['회사명'] = CIS_info_df_combined['회사명'].replace(replace_dict)


In [ ]:
bs_info_df_combined[bs_info_df_combined['회사명'] == '시큐레터(기술특례)']

,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수,재_type
63,시큐레터(기술특례),자산총계,재무상태표,,0,제 9 기말,19518073876,제 10 기 3분기말,13758111075,01579570,3,OFS
64,시큐레터(기술특례),부채총계,재무상태표,,0,제 9 기말,2518140983,제 10 기 3분기말,2072505470,01579570,3,OFS
65,시큐레터(기술특례),자본총계,재무상태표,,0,제 9 기말,16999932893,제 10 기 3분기말,11685605605,01579570,3,OFS
162,시큐레터(기술특례),자산총계,NaN,NaN,0,NaN,0,NaN,0,NaN,3,CFS
163,시큐레터(기술특례),부채총계,NaN,NaN,0,NaN,0,NaN,0,NaN,3,CFS
164,시큐레터(기술특례),자본총계,NaN,NaN,0,NaN,0,NaN,0,NaN,3,CFS


In [ ]:
import pandas as pd
from openpyxl import load_workbook

# 엑셀 파일 경로
excel_path = "/content/drive/MyDrive/webcrawling/경쟁사 실적현황표(SGA.SOL)_24.3Q_V1.xlsx"
new_excel_path = "updated_excel.xlsx"  # 새 파일 경로
# sheet_name = "24.3Q_2411"
sheet_name = "24.4Q_2503"

# 기존 엑셀 파일 불러오기 (서식 유지)
wb = load_workbook(excel_path)
ws = wb[sheet_name]

# 데이터프레임 로드 (bs_info_df, CIS_info_df는 미리 정의되어 있어야 함)
# bs_info_df
# CIS_info_df

# 엑셀의 5행과 21행에 있는 회사명 찾기
target_rows = [4, 23, 42]  # openpyxl은 1-based index (pandas는 0-based index)
for row in target_rows:
    for col in range(1, ws.max_column + 1):  # 모든 열 탐색 (openpyxl은 1부터 시작)
        company_name = ws.cell(row=row, column=col).value  # 회사명 가져오기

        if company_name in bs_info_df_combined["회사명"].values:
          a = ws.cell(row=row-1, column=col).value
          if (a is not None):
            if (a.startswith("*종속")):
              try:
                    # 특정 계정명의 전기금액 & 당기금액 입력 (openpyxl은 1-based index)
                ws.cell(row=row + 3, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "CFS") & (bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자산총계"), "전기금액"].values[0])
                ws.cell(row=row + 4, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "CFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "부채총계"), "전기금액"].values[0])
                ws.cell(row=row + 5, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "CFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자본총계"), "전기금액"].values[0])
                ws.cell(row=row + 3, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "CFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자산총계"), "당기금액"].values[0])
                ws.cell(row=row + 4, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "CFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "부채총계"), "당기금액"].values[0])
                ws.cell(row=row + 5, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "CFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자본총계"), "당기금액"].values[0])

                ws.cell(row=row + 7, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "CFS") &(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "매출액"), "전기누적금액"].values[0])
                ws.cell(row=row + 8, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "CFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "영업이익"), "전기누적금액"].values[0])
                ws.cell(row=row + 10, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "CFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "당기순이익"), "전기누적금액"].values[0])
                ws.cell(row=row + 7, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "CFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "매출액"), "당기누적금액"].values[0])
                ws.cell(row=row + 8, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "CFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "영업이익"), "당기누적금액"].values[0])
                ws.cell(row=row + 10, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "CFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "당기순이익"), "당기누적금액"].values[0])

                ws.cell(row=row + 12, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS") &(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "매출액"), "전기누적금액"].values[0])
                ws.cell(row=row + 13, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "영업이익"), "전기누적금액"].values[0])
                ws.cell(row=row + 12, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "매출액"), "당기누적금액"].values[0])
                ws.cell(row=row + 13, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "영업이익"), "당기누적금액"].values[0])


              except IndexError:
                  print(f"데이터 누락: {company_name}의 일부 값이 없습니다.")
            else:
              try:
                  ws.cell(row=row + 3, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") & (bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자산총계"), "전기금액"].values[0])
                  ws.cell(row=row + 4, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "부채총계"), "전기금액"].values[0])
                  ws.cell(row=row + 5, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자본총계"), "전기금액"].values[0])
                  ws.cell(row=row + 3, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자산총계"), "당기금액"].values[0])
                  ws.cell(row=row + 4, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "부채총계"), "당기금액"].values[0])
                  ws.cell(row=row + 5, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자본총계"), "당기금액"].values[0])

                  ws.cell(row=row + 7, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS") &(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "매출액"), "전기누적금액"].values[0])
                  ws.cell(row=row + 8, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "영업이익"), "전기누적금액"].values[0])
                  ws.cell(row=row + 10, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "당기순이익"), "전기누적금액"].values[0])
                  ws.cell(row=row + 7, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "매출액"), "당기누적금액"].values[0])
                  ws.cell(row=row + 8, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "영업이익"), "당기누적금액"].values[0])
                  ws.cell(row=row + 10, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "당기순이익"), "당기누적금액"].values[0])
              except IndexError:
                  print(f"데이터 누락: {company_name}의 일부 값이 없습니다.")


          else:
            try:
                ws.cell(row=row + 3, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") & (bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자산총계"), "전기금액"].values[0])
                ws.cell(row=row + 4, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "부채총계"), "전기금액"].values[0])
                ws.cell(row=row + 5, column=col, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자본총계"), "전기금액"].values[0])
                ws.cell(row=row + 3, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자산총계"), "당기금액"].values[0])
                ws.cell(row=row + 4, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "부채총계"), "당기금액"].values[0])
                ws.cell(row=row + 5, column=col + 1, value=bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") &(bs_info_df_combined["회사명"] == company_name) & (bs_info_df_combined["계정명"] == "자본총계"), "당기금액"].values[0])

                ws.cell(row=row + 7, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS") &(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "매출액"), "전기누적금액"].values[0])
                ws.cell(row=row + 8, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "영업이익"), "전기누적금액"].values[0])
                ws.cell(row=row + 10, column=col, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "당기순이익"), "전기누적금액"].values[0])
                ws.cell(row=row + 7, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "매출액"), "당기누적금액"].values[0])
                ws.cell(row=row + 8, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "영업이익"), "당기누적금액"].values[0])
                ws.cell(row=row + 10, column=col + 1, value=CIS_info_df_combined.loc[(CIS_info_df_combined["재_type"] == "OFS")&(CIS_info_df_combined["회사명"] == company_name) & (CIS_info_df_combined["계정명"] == "당기순이익"), "당기누적금액"].values[0])
            except IndexError:
                print(f"데이터 누락: {company_name}의 일부 값이 없습니다.")

# 수정된 데이터를 새 파일로 저장 (서식 유지)
wb.save(new_excel_path)

print(f"업데이트 완료: {new_excel_path} -> '{sheet_name}' 시트에 반영됨 (서식 유지됨)")


업데이트 완료: updated_excel.xlsx -> '24.4Q_2503' 시트에 반영됨 (서식 유지됨)


In [ ]:
bs_info_df_combined.loc[(bs_info_df_combined["재_type"] == "OFS") & (bs_info_df_combined["회사명"] == '쌍용정보통신') & (bs_info_df_combined["계정명"] == "자산총계"), "전기금액"].values

array([168565648025])

In [ ]:
# display(bs_info_df_combined[bs_info_df_combined['회사명'] == '모니터랩'])
display(bs_info_df_combined[bs_info_df_combined['회사명'] == '시큐아이'])
# display(bs_info_df_combined[bs_info_df_combined['회사명'] == '모니터랩'])

,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수,재_type


In [ ]:
bs_info_df_combined[bs_info_df_combined['회사명'] == "시큐센"]

,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수,재_type
60,시큐센,자본총계,재무상태표,,0,제 13 기말,13154432361,제 14 기 3분기말,10938505275,01113499,3,OFS
61,시큐센,부채총계,재무상태표,,0,제 13 기말,7609283308,제 14 기 3분기말,7444418379,01113499,3,OFS
62,시큐센,자산총계,재무상태표,,0,제 13 기말,20763715669,제 14 기 3분기말,18382923654,01113499,3,OFS
159,시큐센,자본총계,NaN,NaN,0,NaN,0,NaN,0,NaN,3,CFS
160,시큐센,부채총계,NaN,NaN,0,NaN,0,NaN,0,NaN,3,CFS
161,시큐센,자산총계,NaN,NaN,0,NaN,0,NaN,0,NaN,3,CFS


In [ ]:
CIS_info_df_combined[CIS_info_df_combined['회사명'] == "시큐센"]

,회사명,계정명,고유번호,재무제표명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수,재_type
60,시큐센,매출액,01113499,포괄손익계산서,,NaN,,NaN,3502733536,11325701043,제 14 기 3분기,4890686806,13338873603,2024,3.0,OFS
61,시큐센,영업이익,01113499,포괄손익계산서,,NaN,,NaN,-718875480,-1520233962,제 14 기 3분기,-434781180,-2346979403,2024,3.0,OFS
62,시큐센,당기순이익,01113499,포괄손익계산서,,NaN,,NaN,3795905,-1374835494,제 14 기 3분기,-435530552,-2215927086,2024,3.0,OFS
159,시큐센,매출액,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,CFS
160,시큐센,영업이익,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,CFS
161,시큐센,당기순이익,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,CFS


In [ ]:
bs_info_df_combined[bs_info_df_combined['회사명'] == '시큐아이'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   회사명     0 non-null      category
 1   계정명     0 non-null      category
 2   재무제표명   0 non-null      object  
 3   전전기명    0 non-null      object  
 4   전전기금액   0 non-null      int64   
 5   전기명     0 non-null      object  
 6   전기금액    0 non-null      int64   
 7   당기명     0 non-null      object  
 8   당기금액    0 non-null      int64   
 9   고유번호    0 non-null      object  
 10  행 개수    0 non-null      int64   
 11  재_type  0 non-null      object  
dtypes: category(2), int64(4), object(6)
memory usage: 1.4+ KB


In [ ]:
import pandas as pd
from openpyxl import load_workbook

# 엑셀 파일 경로
excel_path = "/content/drive/MyDrive/webcrawling/경쟁사 실적현황표(SGA.SOL)_24.3Q_V1.xlsx"
new_excel_path = "updated_excel.xlsx"  # 새 파일 경로
sheet_name = "24.3Q_2411"

# 기존 엑셀 파일 불러오기 (서식 유지)
wb = load_workbook(excel_path)
ws = wb[sheet_name]

# 데이터프레임 로드 (bs_info_df, CIS_info_df는 미리 정의되어 있어야 함)
# bs_info_df
# CIS_info_df

# 엑셀의 5행과 21행에 있는 회사명 찾기
target_rows = [4, 23, 42]  # openpyxl은 1-based index (pandas는 0-based index)
for row in target_rows:
    for col in range(1, ws.max_column + 1):  # 모든 열 탐색 (openpyxl은 1부터 시작)
        company_name = ws.cell(row=row, column=col).value  # 회사명 가져오기
        print(company_name)

구분
단위(백만원)
SGA솔루션즈
None
None
시큐브
None
None
안랩
None
None
모니터랩
None
None
이스트소프트
None
None
윈스
None
None
한국정보인증
None
None
라온시큐어
None
None
파수
None
None
지란지교시큐리티
None
None
구분
단위(백만원)
지니언스
None
None
한컴위드
None
None
드림시큐리티
None
None
케이사인
None
None
샌즈랩
None
None
이글루
None
None
파이오링크
None
None
시큐아이
None
None
시큐센
None
None
시큐레터
None
None
구분
단위(백만원)
SGA
None
None
한글과컴퓨터
None
None
MDS테크
None
None
아이티센
None
None
쌍용정보통신
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [ ]:
company_list = ['샌즈랩(기술특례)', '시큐아이(비상장)', 'SGA']
search_text = '시큐아이'

# 리스트 내 요소 중 search_text를 포함하는 항목 출력
matching_items = [item for item in company_list if search_text in item]

# 결과 출력
for item in matching_items:
    print(item)


시큐아이(비상장)


In [ ]:
company_list = ['샌즈랩(기술특례)', '시큐아이(비상장)', 'SGA']
search_text = '시큐아이'

# 리스트 내 요소 중 search_text를 포함하는지 여부 확인
is_present = any(search_text in item for item in company_list)

# 결과 반환
print(is_present)


True
